In [9]:
import cv2
import numpy as np
from utils.opv import OpvModel
facedetectModel = OpvModel("face-detection-adas-0001",device="CPU", fp="FP32")

In [10]:
# "images" 폴더에서 'friends.jpeg' 이미지 파일을 불러와 이미지 사이즈를 '1200, 710'으로 줄입니다.
# 이미지에서 얼굴 있는 부분을 예측하여 변수 'predictions'에 저장합니다.

friends = cv2.imread("images/dog.jpeg")

friends = cv2.resize(friends,(1200, 710))
predictions = facedetectModel.Predict(friends)

In [11]:
predictions.shape

(1, 1, 200, 7)

In [12]:
# 이미지에서 얼굴로 예측(예측확률 0%보다 큰)된 모든 주변에 경계 상자를 그리는 코드를 완성합니다.

def DrawBoundingBoxes(predictions, image, conf=0):
    canvas = image.copy()
    predictions_1 = predictions[0][0]
    confidence = predictions_1[:,2]
    
    topresults = predictions_1[(confidence > conf)]
    (h, w) = canvas.shape[:2]
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h])
        (xmin, ymin, xmax, ymax) = box.astype("int")
        
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    cv2.putText(canvas, str(len(topresults))+" face(s) detected", (50,50),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    return canvas

cv2.imshow("dog", DrawBoundingBoxes(predictions, friends))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
# 이미지에서 얼굴로 예측(예측 확률 50% 보다 큰)된 모든 주변에 경계 상자를 그리는 코드를 완성

def DrawBoundingBoxes(predictions, image, conf = 0.5):
    canvas = image.copy()
    predictions_1 = predictions[0][0]
    confidence = predictions_1[:,2]
    
    topresults = predictions_1[(confidence>conf)]
    (h,w) = canvas.shape[:2]
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h])
        (xmin, ymin, xmax, ymax) = box.astype("int")
        
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    cv2.putText(canvas, str(len(topresults))+" face(s) detected", (50,50),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    return canvas

cv2.imshow("dog",DrawBoundingBoxes(predictions,friends))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
# 웹캠을 이용하여 실시간 영상에서 얼굴은 인식하고
# 얼굴 주변에 상자의 색상과 두께, 글꼴의 크기와 유형을 변경하는 코드를 완성합니다.

from utils.opv import OpvModel
import cv2
import numpy as np

facedetectModel = OpvModel("face-detection-adas-0001", "CPU", "FP32")
camera = cv2.VideoCapture(0)

def DrawBoundingBoxes(predictions, image, conf=0.5):
    canvas = image.copy()
    predictions = predictions[0][0]
    confidence = predictions[:,2]
    
    topresults = predictions[(confidence>conf)]
    (h,w) = canvas.shape[:2]
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h])
        (xmin, ymin, xmax, ymax) = box.astype("int")
        
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0, 0, 255), 4)
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    cv2.putText(canvas, str(len(topresults))+" face(s) detected", (50,50),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0,0), 2)
    return canvas

while(True):
    ret, frame = camera.read() # Capture frame by frame
    if ret == False:
        break
    predictions = facedetectModel.Predict(frame)
    
    try:
        cv2.imshow('Press Spacebar to Exit', DrawBoundingBoxes(predictions, frame))
    except Exception as e:
        print(e)
        break
        
    if cv2.waitKey(1) & 0xFF == ord(' '): # Stop if spacebar is detected
        break
        
camera.release() # Cleanup after spacebar is detected.
cv2.destroyAllWindows()